# Web Scrapping: Normativa en trámite con vigencia para observaciones (SUSESO)

El siguiente código realiza una revisión y extracción del código fuente de la página de SUSESO. Los requisitos para poder ejecutar este código son:

 - **Instalar librerías:**
  - pandas
  - scrapy
  - requests
  - datetime
  - wget
 - **Tener creadas las carpetas para almacenar los archivos.**
  - En el **código 18** está la ruta en donde se almacena la información, las cuales deben ser cambiadas por quien ejecute el código para que referencien a su propio equipo.
  
---
 - **Fecha de creación: 08.10.2020 (IIC)** 

## Importar librerías

In [1]:
import pandas as pd
from scrapy import Selector
import requests
from datetime import datetime, timedelta
import wget

## Cargar código html a utilizar

In [2]:
html = requests.get('https://www.suseso.cl/612/w3-propertyvalue-34035.html').content

### Crear selectors para después buscar la información

In [3]:
sel = Selector(text = html)

## Búsqueda de información

### Lista fecha publicación

En la siguiente línea de código se busca el tag "td" con la clase *fecha periodo* y se extrae el texto asociado.

In [4]:
fecha_publicacion = sel.xpath('//td[contains(@class,"fecha cid-512 pnid-524")]/text()').extract()

### Lista Detalle

En la siguiente línea de código se busca el tag "td" que contenga una clase en particular en todo el código y se extrae el texto asociado.

In [5]:
materia = sel.xpath('//td[contains(@class," cid-512 pnid-525 pn-attr")]/text()').extract()

### Lista fecha para observaciones

En la siguiente línea de código se busca el tag "td" que contenga una clase en particular en todo el código y se extrae el texto asociado.

In [6]:
fecha_observacion = sel.xpath('//td[contains(@class,"fecha cid-512 pnid-555")]/text()').extract()

### Lista email para observaciones

En la siguiente línea de código se busca el tag "td" que contenga una clase en particular en todo el código y se extrae el texto asociado.

In [7]:
email_observacion = sel.xpath('//td[contains(@class,"cid-512 pnid-554")]/text()').extract()

### Lista con links

En la siguiente línea de código se busca en todos los tag td que tengan como "hijo" un tag span con la clase "td_20_porciento cid-19" y este a su vez, tenga asociado un hipervículo el cual se extrae.

In [8]:
links = sel.xpath('//td[contains(@class,"normativa-consultar")]/span[contains(@class,"figure cid-512")]/a/@href').extract()

In [9]:
urls = []

for i in links:
    url = 'https://www.suseso.cl/612/' + i
    urls.append(url)

## Almacenar información obtenida y agregar formatos

In [10]:
df_norm_tramite = pd.DataFrame({
    'Fecha Publicación': fecha_publicacion,
    'Materia': materia,
    'Fecha envio observaciones': fecha_observacion,
    'Mail envio observaciones': email_observacion,
    'Link': urls})

In [11]:
#Creamos nueva columna con fecha publicación y observación.
df_norm_tramite['Titulo'] =  df_norm_tramite['Fecha Publicación'] + "_" + df_norm_tramite['Fecha envio observaciones']

In [12]:
#Aplicamos formato fecha a columna "Fecha Publicación" y "Fecha envio observaciones"
df_norm_tramite['Fecha Publicación'] = pd.to_datetime(
    df_norm_tramite['Fecha Publicación'],
    format = '%d/%m/%Y')

In [13]:
df_norm_tramite

,Fecha Publicación,Materia,Fecha envio observaciones,Mail envio observaciones,Link,Titulo
0,2020-10-05,Actualización manual del método del cuestionar...,16/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-605591_arch...,05/10/2020_16/10/2020
1,2020-10-02,Evaluación de vigilancia ambiental y de la sal...,20/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-605477_arch...,02/10/2020_20/10/2020
2,2020-10-02,Guía para la aplicación del Estudio de Puesto ...,16/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-605480_arch...,02/10/2020_16/10/2020
3,2020-09-28,Proyecto de Circular - Plan de Prevención de A...,16/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-604531_arch...,28/09/2020_16/10/2020
4,2020-09-28,Portabilidad Financiera y Régimen de Crédito S...,05/10/2020,normativoibs@suseso.cl,https://www.suseso.cl/612/articles-604530_arch...,28/09/2020_05/10/2020
5,2020-09-14,Proyecto Circular - Capacitaciones de los cola...,05/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-602804_arch...,14/09/2020_05/10/2020


## Filtrar novedades

In [14]:
novedad = datetime.now() - timedelta(days=60)

In [15]:
df_norm_novedad = df_norm_tramite.loc[(df_norm_tramite['Fecha Publicación'] >= novedad)]
df_norm_novedad

,Fecha Publicación,Materia,Fecha envio observaciones,Mail envio observaciones,Link,Titulo
0,2020-10-05,Actualización manual del método del cuestionar...,16/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-605591_arch...,05/10/2020_16/10/2020
1,2020-10-02,Evaluación de vigilancia ambiental y de la sal...,20/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-605477_arch...,02/10/2020_20/10/2020
2,2020-10-02,Guía para la aplicación del Estudio de Puesto ...,16/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-605480_arch...,02/10/2020_16/10/2020
3,2020-09-28,Proyecto de Circular - Plan de Prevención de A...,16/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-604531_arch...,28/09/2020_16/10/2020
4,2020-09-28,Portabilidad Financiera y Régimen de Crédito S...,05/10/2020,normativoibs@suseso.cl,https://www.suseso.cl/612/articles-604530_arch...,28/09/2020_05/10/2020
5,2020-09-14,Proyecto Circular - Capacitaciones de los cola...,05/10/2020,isesat@suseso.cl,https://www.suseso.cl/612/articles-602804_arch...,14/09/2020_05/10/2020


In [16]:
nombre_archivo = df_norm_novedad['Titulo']
nombre_archivo = nombre_archivo.str.replace('/','-')

In [17]:
urls_novedad = df_norm_novedad['Link']

## Descargar novedades

### Rutas para almacenar la información

In [18]:
paths = []

for i in nombre_archivo:
    # La ruta escrita entre '' debe ser cambiada por quien ejecute el código.
    path = 'C:/projects/datalake/data/output/current/cambios_normativos/suseso/normativa_tramite/' + i
    paths.append(path)

### Descargar archivos

In [19]:
i = 0

for url in urls_novedad:
    wget.download(url,paths[i]+'.pdf')
    i = i + 1

### Guardar DF históricos en Excel

In [20]:
# Eliminamos columnas auxiliares
df_norm_tramite = df_norm_tramite.drop(['Titulo'], axis=1)

In [21]:
history_path = 'C:/projects/datalake/data/output/history/cambios_normativos/suseso/normativa_tramite/'
nombre_archivo = 'Normativa_trámite(en_plazo)'
df_norm.to_csv(output_path + datetime.now().strftime('%Y%m%d') + nombre_archivo + '.csv')